In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# 假设你有一个轨迹数据集和对应的特征点标注
train_trajectories = ...  # shape: (num_samples, sequence_length, input_dim)
feature_points = ...  # shape: (num_samples, sequence_length)

# 定义模型
class TrajectoryModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TrajectoryModel, self).__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, h = self.rnn(x)
        out = self.fc(h.squeeze(0))
        return out

input_dim = train_trajectories.shape[-1]
hidden_dim = 128
output_dim = 1  # 输出为特征点的概率

model = TrajectoryModel(input_dim, hidden_dim, output_dim)

# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 开始训练
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0

    for trajectory, feature_point in zip(train_trajectories, feature_points):
        optimizer.zero_grad()

        # 将特征点作为Prompt插入到轨迹数据中
        trajectory_with_prompt = torch.cat((trajectory, feature_point.unsqueeze(-1)), dim=-1)

        # 前向传播
        outputs = model(trajectory_with_prompt)

        # 计算损失
        loss = criterion(outputs, feature_point.float())

        # 反向传播和优化
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_trajectories)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# 训练完成后，可以使用训练好的模型进行预测和评估



AttributeError: 'ellipsis' object has no attribute 'shape'

In [4]:
import numpy as np

def eucl_dist(x,y):
    """
    Usage
    -----
    L2-norm between point x and y
    Parameters
    ----------
    param x : numpy_array
    param y : numpy_array
    Returns
    -------
    dist : float
           L2-norm between x and y
    """
    dist = np.linalg.norm(x-y)
    return dist

def _c(ca,i,j,P,Q):
    if ca[i,j] > -1:
        return ca[i,j]
    elif i == 0 and j == 0:
        ca[i,j] = eucl_dist(P[0],Q[0])
    elif i > 0 and j == 0:
        ca[i,j] = max(_c(ca,i-1,0,P,Q),eucl_dist(P[i],Q[0]))
    elif i == 0 and j > 0:
        ca[i,j] = max(_c(ca,0,j-1,P,Q),eucl_dist(P[0],Q[j]))
    elif i > 0 and j > 0:
        ca[i,j] = max(min(_c(ca,i-1,j,P,Q),_c(ca,i-1,j-1,P,Q),_c(ca,i,j-1,P,Q)),eucl_dist(P[i],Q[j]))
    else:
        ca[i,j] = float("inf")
    return ca[i,j]


def discret_frechet(P,Q):
    """
    Usage
    -----
    Compute the discret frechet distance between trajectories P and Q
    Parameters
    ----------
    param P : px2 numpy_array, Trajectory P
    param Q : qx2 numpy_array, Trajectory Q
    Returns
    -------
    frech : float, the discret frechet distance between trajectories P and Q
    """
    ca = np.ones((len(P),len(Q)))
    ca = np.multiply(ca,-1)
    return _c(ca,len(P)-1,len(Q)-1,P,Q)